# TITLE: Boosting Paper Relication 
### Purpose: 
To replcate the paper "Boosting Automatic Commit Classification Into Maintenance Activites By Utilizing Source Code Changes" by **Stanislav Levin** and **Amiram Yehudai**

In [61]:
import pandas as pd 
import numpy as np 
import psycopg2 
import xlrd 
import os

In [62]:
path_file = os.getcwd()
print(path_file)

/home/kelechi/Documents/ML_Auto_Tagger


In [63]:
df = pd.read_excel(path_file + '/Commit data.xlsx')

In [64]:
df.head()

,commit id,class
0,0b9ea98,NFT
1,980d9f8,RFT
2,e9d5060,NFT
3,a63639b,NFT
4,a611d64,RMN


In [65]:
df.shape

(926, 2)

In [66]:
try: 
    conn = psycopg2.connect("dbname='squad' user='postgres' password='SolomonLaine_11251994' host='localhost'")
except Exception as e: 
    print(e)
cur = conn.cursor()

In [67]:
csha_list = df['commit id'].values 

In [68]:
csha_list[0]

'0b9ea98'

In [69]:
commit_message = []
commits_no_message = []
for index, csha in enumerate(csha_list):
    qr = "SELECT message FROM commits WHERE csha LIKE '{}%'".format(csha_list[index].strip())
    cur.execute(qr)
    fetched_row = cur.fetchone()
    if fetched_row != None: 
        commit_message.append(fetched_row[0])
    else:
        print('None')
        commit_message.append('None')
        commits_no_message.append(csha)

None
None
None


In [70]:
print(commits_no_message)
df['commit_message'] = commit_message

['9dcf08d', '86b3e40', '917af3e']


In [71]:
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kelechi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/kelechi/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kelechi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
from ast import literal_eval
import re

##### (i) Remove git-svn-id: 
e.g. git-svn-id: https://svn.apache.org/repos/asf/commons/proper/collections/trunk@1492866 13f79535-47bb-0310-9956-ffa450edef68

##### (ii) Remove Issue Number : e.g. [CALCITE-968]  [COMPRESS-392]

##### (iii) Remove Created by MOE - 
eg. Created by MOE: https://github.com/google/moe
MOE_MIGRATED_REVID=160142836

##### (iv) Remove Developer name
e.g. 344 [] User-defined aggregate functions with more than one parameter (hzyuemeng1)

##### (v) Remove http and email address

##### (vi) Other filter

- Authors
- Reviewers
- [...]
- digitial
- special character


In [74]:
# words in custom_words_list are already stemmed 
custom_words_list = ['patch', 'hbase', 'checksum', 'code', 'version', 'byte', 'data', 'hfile', 'region', 'schedul', 'singl', 'can',
                'yarn', 'contribut', 'commit', 'merg', 'make', 'trunk', 'hadoop', 'svn', 'ignoreancestri', 'node', 'also', 
                'client', 'hdfs', 'mapreduc', 'lipcon', 'idea', 'common', 'file', 'ideadev', 'plugin', 'project', 'modul', 
                'find', 'border', 'addit', 'changeutilencod', 'clickabl', 'color', 'column', 'cach', 'jbrule', 'drool', 'coprocessor', 'regionserv', 
                'scan', 'resourcemanag', 'cherri', 'gong', 'ryza', 'sandi', 'xuan', 'token', 'contain', 'shen', 'todd', 'zhiji', 'tan', 
                'wangda', 'timelin', 'app', 'kasha', 'kashacherri', 'messag', 'spr', 'camel', 'http', 'now', 'class', 'default', 
                'pick', 'via']
custom_words_set = set(custom_words)

In [75]:
GIT_SVN_RE = re.compile('git-svn-id:.*\n')
issue_number_re = re.compile('[A-Za-z]+[-]\d+')
created_by_moe = re.compile('Created by MOE:.*\n')
MOE_ID = re.compile('MOE_MIGRATED_REVID=.*\n')
developer_name_re = re.compile('\(.*\)\n')
HTTP_RE = re.compile('(http|https):.*\n')
EMAIL_RE = re.compile('[a-zA-Z0-9]+@[a-zA-Z0-9]+\.[a-zA-Z0-9-.]+')
Other_RE = re.compile('(Author|Authors|Reviewers|Reviewer).*\n')
Bracket_RE = re.compile('\[.*\]')

break_line_re = re.compile('\n')
digital = re.compile('[0-9]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z ]') 
STOPWORDS = set(stopwords.words('english'))

In [76]:
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = GIT_SVN_RE.sub('',text)
    text = issue_number_re.sub('',text)
    text = created_by_moe.sub('',text)
    text = MOE_ID.sub('',text)
    text = developer_name_re.sub('\n', text)
    text = HTTP_RE.sub('\n', text)
    text = EMAIL_RE.sub('\n',text)
    text = Other_RE.sub('', text)
    text = Bracket_RE.sub('', text)
    
    text = text.lower() # lowercase text
    text = break_line_re.sub(' ',text) # remove all \n
    text = digital.sub('',text)
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS]) # delete stopwords from text
    return text

In [77]:
test_commit = df['commit_message'].values[232]
print(test_commit)
test_commit = text_prepare(test_commit)
print(test_commit)

Reconcile the cancellation semantics for Futures.combine, successfulAsList, allAsList, and withFallback to be consistent with Futures.transform.  Now, all cancellations are back-propagated to the input futures.  We also expose the interruption state in AbstractFuture with a protected method.  To ensure the interruption state is correct and threadsafe, we add an additional state to AbstractFuture's inner Sync class to atomically distinguish between cancelled and interrupted.
-------------
Created by MOE: http://code.google.com/p/moe-java
MOE_MIGRATED_REVID=33468640

reconcile cancellation semantics futurescombine successfulaslist allaslist withfallback consistent futurestransform cancellations backpropagated input futures also expose interruption state abstractfuture protected method ensure interruption state correct threadsafe add additional state abstractfutures inner sync class atomically distinguish cancelled interrupted


In [78]:
df['commit_message'] = [text_prepare(line) for line in df['commit_message']]
for index, line in enumerate(df['commit_message'].values):
    if index < 10:
        print(index, line)

0 add timestampadd timestampdiff functions close apachecalcite
1 move code jdbcimplementor jdbcrules new class sqlimplementor
2 streamtorelation streamtostream joins rule transform deltascanconstanttable empty fix nullpointerexception pruneemptyrules
3 add avatica openconnectionrequest goal passing connection properties info remote avatica driver corresponding serverside connection changes include meta explicit opening connections new openconnection call client decides connection id implicit creation unknown connections gone jdbcmeta default connection gone methods extended connection id correspondingly various servicerequests carry connection id unrelated useful testing meaningful error messages remote driver
4 reorganize package structure naming convention planner rules change renames files deletes obsolete files content changes
5 infer collation project using monotonicity close apacheincubatorcalcite
6 add piglet subset pig latin top calcite algebra includes parser distinct order li

## SpellChecking 
Three Options:

    1. keep all wrong words
    2. remove all wrong words
    3. replace wrong words with some new words


In [79]:
!pip3 install pyenchant

In [80]:
from enchant.checker import SpellChecker
chkr = SpellChecker("en_US")
wrong_words = []
for index, text in enumerate(df['commit_message']):
    chkr.set_text(text)
    for err in chkr:
        if err.word not in wrong_words:
            wrong_words.append(err.word)
        #   print(index, err.word)
        
def remove_noise_words(message_array):
    res = []
    for text in message_array:
        text = ' '.join([x for x in text.split() if x and x not in wrong_words])
        res.append(text)
    return res

def replace_nosie_words_with_sth(message_array):
    res = []
    for text in message_array:
        # replace wrong word with something
        new_text = []
        for x in text.split():
            if x and x in wrong_words:
                new_text.append('something')
            else:
                new_text.append(x)
                
        text = ' '.join(new_text)
        res.append(text)
    return res

remove_noise_message = remove_noise_words(df['commit_message'].values)
for index, line in enumerate(remove_noise_message):
    if index < 10:
        print(index, line)

0 add functions close
1 move code new class
2 joins rule transform empty fix
3 add goal passing connection properties info remote driver corresponding connection changes include meta explicit opening connections new call client decides connection id implicit creation unknown connections gone default connection gone methods extended connection id correspondingly various carry connection id unrelated useful testing meaningful error messages remote driver
4 reorganize package structure naming convention planner rules change renames files deletes obsolete files content changes
5 infer collation project using close
6 add piglet subset pig top calcite algebra includes parser distinct order limit group commands incomplete nested command add tests based examples reference
7 revert changes introduced rev particular means package coordinates changed back furthermore maven coordinates changes back
8 move methods correct class
9 bug select instructions implement instead


In [81]:
replaced_message = replace_nosie_words_with_sth(df['commit_message'].values)
for index, line in enumerate(replaced_message):
    if index < 10:
        print(index, line)

0 add something something functions close something
1 move code something something new class something
2 something something joins rule transform something empty fix something something
3 add something something goal passing connection properties info remote something driver corresponding something connection changes include meta explicit opening connections new something call client decides connection id implicit creation unknown connections gone something default connection gone methods extended connection id correspondingly various something carry connection id unrelated useful testing meaningful error messages remote driver
4 reorganize package structure naming convention planner rules change renames files deletes obsolete files content changes
5 infer collation project using something close something
6 add piglet subset pig something top calcite algebra includes parser distinct order limit group commands incomplete nested something command add tests based examples something refer

In [82]:
original_commit_message = np.copy(df['commit_message'])

In [95]:
df['commit_message'] = remove_noise_message

## Word Stemming 

In [96]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english") # Choose a language
# stemmer.stem("countries")
for index, text in enumerate(df['commit_message']):
#     print(index, df['cmt_msg'][index])
    df['commit_message'][index] = ' '.join([stemmer.stem(x) for x in text.split()])
#     print(index, df['cmt_msg'][index])

In [97]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
# for index, text in enumerate(df['cmt_msg']):
for i in range(len(df['commit_message'])):
    print(i, df['commit_message'][i])
    df['commit_message'][i] = ' '.join([wnl.lemmatize(x) for x in df['commit_message'][i].split()])
    print(i, df['commit_message'][i])

0 add function close
0 add function close
1 move code new class
1 move code new class
2 join rule transform empti fix
2 join rule transform empti fix
3 add goal pass connect properti info remot driver correspond connect chang includ meta explicit open connect new call client decid connect id implicit creation unknown connect gone default connect gone method extend connect id correspond various carri connect id unrel use test meaning error messag remot driver
3 add goal pas connect properti info remot driver correspond connect chang includ meta explicit open connect new call client decid connect id implicit creation unknown connect gone default connect gone method extend connect id correspond various carri connect id unrel use test meaning error messag remot driver
4 reorgan packag structur name convent planner rule chang renam file delet obsolet file content chang
4 reorgan packag structur name convent planner rule chang renam file delet obsolet file content chang
5 infer collat projec

83 creat test framework requir format
83 creat test framework requir format
84 add miss file
84 add miss file
85 disabl compress option
85 disabl compress option
86 commit patch
86 commit patch
87 ad spill queue
87 ad spill queue
88 improv
88 improv
89 fill counter use inform
89 fill counter use inform
90 remov useless code
90 remov useless code
91 block job job
91 block job job
92 add licens
92 add licens
93 flume support expos metric via web servic format
93 flume support expo metric via web servic format
94 implement
94 implement
95 silent fail first transact channel
95 silent fail first transact channel
96 none
96 none
97 delet deprec code
97 delet deprec code
98 various fix fix potenti reduc visibl mani method use signatur null argument need one patch dot com
98 various fix fix potenti reduc visibl mani method use signatur null argument need one patch dot com
99 constructor wrap allow cach interpret overload discoveri relat constructor
99 constructor wrap allow cach interpret over

379 fix syntax seem allow generic type use entiti simul result neat hack
380 fix warn relat
380 fix warn relat
381 statement unnecessarili nest within els claus
381 statement unnecessarili nest within el claus
382 ad
382 ad
383 document size wrap collect releas leav would requir much work improv later
383 document size wrap collect releas leav would requir much work improv later
384 explicit
384 explicit
385 remov unneed import
385 remov unneed import
386 remov unneed tag format
386 remov unneed tag format
387 ad sinc tag fix exampl
387 ad sinc tag fix exampl
388 add miss sinc tag
388 add miss sinc tag
389 remov type review
389 remov type review
390 make field privat final possibl
390 make field privat final possibl
391 fix compil error java thank
391 fix compil error java thank
392 revert commit
392 revert commit
393 add support archiv header stream filter extend
393 add support archiv header stream filter extend
394 provid read access metadata
394 provid read access metadata
395 unne

504 add file invert
505 client call logout disconnect
505 client call logout disconnect
506 fix class extend take constructor
506 fix class extend take constructor
507 add
507 add
508 fix
508 fix
509 fix issu
509 fix issu
510 final
510 final
511 clean
511 clean
512 add version support
512 add version support
513 improv plugin use core handler improv group map ad liberti support ad exampl
513 improv plugin use core handler improv group map ad liberti support ad exampl
514 minor fix
514 minor fix
515 fix metadata
515 fix metadata
516 rewrit oper operand prune work correct
516 rewrit oper operand prune work correct
517 creat object refactor introduc pagin mechan
517 creat object refactor introduc pagin mechan
518 updat implement use new metadata updat driver use metadata instead execut queri close
518 updat implement use new metadata updat driver use metadata instead execut queri close
519 fix select root singl top level partit close
519 fix select root singl top level partit close
520 fi

759 remov dead code
759 remov dead code
760 soy remov renam move anoth packag forgot p rm manag class rollback need manag class dead code
760 soy remov renam move anoth packag forgot p rm manag class rollback need manag class dead code
761 soy extract nice featur grok print snippet sourc code associ node edg exampl xref code search one way provid snippet would call node howev guarante actual sourc code would probabl get confus exact snippet logic need cl extract new class sinc snippet use error
761 soy extract nice featur grok print snippet sourc code associ node edg exampl xref code search one way provid snippet would call node howev guarante actual sourc code would probabl get confus exact snippet logic need cl extract new class sinc snippet use error
762 chang increment code gen proper handl void tag ad call close form open tag declar chang close node visitor ignor void element
762 chang increment code gen proper handl void tag ad call close form open tag declar chang close node vis

## Keywords Distribution 
##### (i) Removed custom words: 
e.g ('patch', 'hbase', 'checksum', 'code', 'version', 'byte', 'data', 'hfile', 'region', 'schedul', 'singl', 'can',
'yarn', 'contribut', 'commit', 'merg', 'make', 'trunk', 'hadoop', 'svn', 'ignoreancestri', 'node', 'also', 'client', 'hdfs', 'mapreduc', 'lipcon', 'idea', 'common', 'file', 'ideadev', 'plugin', 'project', 'modul', 'find', 'border', 'addit', 'changeutilencod', 'clickabl', 'color', 'column', 'cach', 'jbrule', 'drool', 'coprocessor', 'regionserv', 'scan', 'resourcemanag', 'cherri', 'gong', 'ryza', 'sandi', 'xuan', 'token', 'contain', 'shen', 'todd', 'zhiji', 'tan', 'wangda', 'timelin', 'app', 'kasha', 'kashacherri', 'messag', 'spr', 'camel', 'http', 'now', 'class', 'default', 'pick', 'via')

##### (ii) We selected 51 most frequent words per class (p.s threshold variable) 

In [98]:
from scipy import sparse as sp_parse 
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter 

In [99]:
words_counts = Counter()

for words in remove_noise_message:
    for word in words.split():
        words_counts[word] += 1

most_common_words_without_noise = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:20]

print('Most common words total:')
for word in most_common_words_without_noise:
    print(word)

Most common words total:
('add', 125)
('fix', 95)
('added', 79)
('use', 74)
('code', 67)
('methods', 62)
('support', 61)
('method', 59)
('remove', 58)
('test', 50)
('new', 48)
('change', 47)
('fixed', 47)
('classes', 41)
('class', 40)
('make', 40)
('type', 33)
('changes', 32)
('comments', 32)
('used', 32)


In [100]:
tags_counts = Counter()
X, y = df['commit_message'].values, df['class'].values
for tags in y:
    tags_counts[tags] += 1
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:-1]
most_common_tags

[('NFT', 276),
 ('FIX', 193),
 ('MOD', 156),
 ('DOC', 53),
 ('RFT', 43),
 ('DEL', 41),
 ('RMN', 37),
 ('CLN', 31),
 ('VER', 25),
 ('DPD', 21),
 ('ANT', 17),
 ('CMT', 15),
 ('TST', 7),
 ('IMP', 6),
 ('BRC', 4)]

In [101]:
classes_ = df['class'].value_counts().index

In [102]:
most_frequent_words_cat = dict()   
for cl in classes_:
    print("=====================")
    print(cl)
    commit_messages_cl = df[df['class'] == cl]['commit_message'].values
    
    words_counts = Counter()
    
    for words in commit_messages_cl:
        hash_dict = dict()
        for word in words.split():
            if word not in hash_dict and custom_words_set:  # remove VCS words in boosting paper 
                hash_dict[word] = True
                words_counts[word] += 1
            else:
                pass
    
    most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:300]
    most_frequent_words_cat[cl] = most_common_words
    
    for word in most_common_words:
        print(word)
        
for key, vals in most_frequent_words_cat.items(): 
    print(key)
    for val in vals: 
        print(val)

NFT
('add', 77)
('use', 43)
('support', 41)
('ad', 32)
('implement', 30)
('method', 29)
('chang', 24)
('allow', 18)
('code', 17)
('new', 16)
('class', 15)
('patch', 14)
('close', 13)
('fix', 13)
('test', 13)
('file', 13)
('function', 12)
('provid', 12)
('option', 12)
('properti', 11)
('error', 11)
('remov', 11)
('check', 11)
('default', 10)
('interfac', 10)
('type', 10)
('state', 10)
('make', 10)
('call', 9)
('base', 9)
('user', 9)
('write', 9)
('one', 9)
('also', 9)
('client', 8)
('appli', 8)
('creat', 8)
('format', 8)
('get', 8)
('command', 7)
('exampl', 7)
('updat', 7)
('return', 7)
('list', 7)
('handl', 7)
('filter', 7)
('generat', 7)
('issu', 7)
('log', 7)
('messag', 6)
('inform', 6)
('object', 6)
('node', 6)
('initi', 6)
('commit', 6)
('index', 6)
('see', 6)
('line', 6)
('pas', 5)
('connect', 5)
('includ', 5)
('id', 5)
('extend', 5)
('parser', 5)
('event', 5)
('configur', 5)
('key', 5)
('requir', 5)
('multipl', 5)
('par', 5)
('name', 5)
('encod', 5)
('map', 5)
('case', 5)
('rule'

('cur', 1)
('brace', 1)
('import', 1)
('move', 1)
('optim', 1)
('specif', 1)
('new', 1)
('class', 1)
('allow', 1)
('r', 1)
('broken', 1)
('link', 1)
('googl', 1)
('issu', 1)
('tracker', 1)
('replac', 1)
('mirror', 1)
('redund', 1)
('call', 1)
('small', 1)
('super', 1)
('return', 1)
('actual', 1)
('provid', 1)
('addit', 1)
('user', 1)
('mild', 1)
('annoy', 1)
('line', 1)
('recommend', 1)
('avoid', 1)
('wildcard', 1)
('almost', 1)
('never', 1)
('need', 1)
('put', 1)
('burden', 1)
('autom', 1)
('intern', 1)
('tool', 1)
('paramet', 1)
('caller', 1)
('explicit', 1)
('specifi', 1)
('inadvert', 1)
('check', 1)
('error', 1)
VER
('fix', 6)
('use', 5)
('version', 5)
('upgrad', 4)
('java', 4)
('updat', 4)
('valu', 2)
('new', 2)
('chang', 2)
('test', 2)
('fail', 2)
('error', 2)
('add', 2)
('compil', 2)
('set', 2)
('default', 1)
('schema', 1)
('actual', 1)
('disabl', 1)
('compress', 1)
('option', 1)
('artifact', 1)
('code', 1)
('move', 1)
('project', 1)
('see', 1)
('bug', 1)
('report', 1)
('detail'

('align', 1)
('pom', 1)
('clear', 1)
('identifi', 1)
('java', 1)
('reformat', 1)
('simpl', 1)
('factor', 1)
('logic', 1)
('current', 1)
('well', 1)
('part', 1)
('sinc', 1)
('yet', 1)
('migrat', 1)
('pas', 1)
('futur', 1)
('travers', 1)
('two', 1)
('portion', 1)
('like', 1)
('bodi', 1)
('control', 1)
('flow', 1)
('structur', 1)
('assign', 1)
('unnecessari', 1)
('side', 1)
('necessari', 1)
('unrel', 1)
('pull', 1)
('rewrit', 1)
('helper', 1)
('static', 1)
('cleanup', 1)
('oper', 1)
('merg', 1)
('push', 1)
('retain', 1)
('effect', 1)
('byte', 1)
('easier', 1)
('run', 1)
DEL
('remov', 24)
('code', 8)
('deprec', 7)
('method', 6)
('delet', 6)
('class', 6)
('replac', 3)
('new', 3)
('relat', 3)
('use', 3)
('implement', 3)
('unus', 3)
('call', 2)
('packag', 2)
('none', 2)
('move', 2)
('manag', 2)
('default', 2)
('make', 2)
('add', 2)
('longer', 2)
('run', 2)
('renam', 2)
('enabl', 2)
('soy', 2)
('dead', 2)
('charact', 1)
('set', 1)
('encod', 1)
('special', 1)
('case', 1)
('malform', 1)
('input'

In [103]:
def get_class_length(dictionary_obj):
    '''
    return:  returns dictionary with class lengths 
    '''
    class_length = dict()  # length of dictionary values in most_frequent_words_cat 
    for key, vals in most_frequent_words_cat.items(): 
        count = 0
        for val in vals: 
            count += 1 
        class_length[key] = count
    return class_length 

# smallest class 'MDL'

In [104]:
threshold = 51 # threshold for the number of keywords 
for key, vals in most_frequent_words_cat.items(): 
    temp_list = []
    if len(vals) > threshold: 
        temp_list = vals[:threshold]
    else:
        temp_list = vals 
    most_frequent_words_cat[key] = temp_list
    
class_length = get_class_length(most_frequent_words_cat)
for key, val in class_length.items(): 
    print(key, val)


NFT 51
FIX 51
MOD 51
DOC 51
RFT 51
DEL 51
RMN 51
CLN 51
VER 51
DPD 51
ANT 51
CMT 51
TST 47
IMP 10
BRC 42
MDL 2


## TRIMMING FREQUENT WORDS

In [105]:
temp_list = []
for key, vals in most_frequent_words_cat.items(): 
    for val, _ in vals: 
        temp_list.append(val)
unique_keywords_words_cat = set(temp_list)
print(len(unique_keywords_words_cat))
print(" ")

for val in unique_keywords_words_cat:
    print(val)


352
 
licens
actual
invok
select
java
tri
format
connect
read
prevent
number
dot
could
addit
warn
report
built
argument
manag
appear
current
master
class
exampl
user
caus
convent
pool
target
scenario
want
activ
filter
object
planner
flume
instanc
soon
henc
contribut
reorgan
impli
incorrect
includ
integr
authent
sourc
packag
simplifi
incomplet
behind
good
generic
describ
chang
key
drop
immedi
return
default
case
review
techniqu
inform
appropri
author
coordin
phoenix
import
valu
call
list
initi
compress
skylark
recov
upgrad
special
scheme
comment
suit
declar
compat
usag
way
creat
client
may
need
constant
issu
imposs
test
soy
definit
thing
indent
framework
would
appli
reduc
renam
final
privat
trivial
ensur
affect
attempt
mean
effici
messag
match
alway
type
remain
array
break
method
consum
add
inaccur
artifact
extend
base
correct
timestamp
applic
specif
rev
compil
space
typo
malform
see
disabl
valid
none
parser
command
updat
enough
line
bug
data
revert
get
separ
tradit
stream
instead
quest

In [109]:
key_words_features = np.zeros((926, len(unique_keywords_words_cat)))
unique_keywords_words_cat = list(unique_keywords_words_cat)
key_words_features_dict = dict()
for val in unique_keywords_words_cat: 
    key_words_features_dict[val] = unique_keywords_words_cat.index(val)

for key, val in key_words_features_dict.items(): 
    print(key, val)

licens 0
actual 1
invok 2
select 3
java 4
tri 5
format 6
connect 7
read 8
prevent 9
number 10
dot 11
could 12
addit 13
warn 14
report 15
built 16
argument 17
manag 18
appear 19
current 20
master 21
class 22
exampl 23
user 24
caus 25
convent 26
pool 27
target 28
scenario 29
want 30
activ 31
filter 32
object 33
planner 34
flume 35
instanc 36
soon 37
henc 38
contribut 39
reorgan 40
impli 41
incorrect 42
includ 43
integr 44
authent 45
sourc 46
packag 47
simplifi 48
incomplet 49
behind 50
good 51
generic 52
describ 53
chang 54
key 55
drop 56
immedi 57
return 58
default 59
case 60
review 61
techniqu 62
inform 63
appropri 64
author 65
coordin 66
phoenix 67
import 68
valu 69
call 70
list 71
initi 72
compress 73
skylark 74
recov 75
upgrad 76
special 77
scheme 78
comment 79
suit 80
declar 81
compat 82
usag 83
way 84
creat 85
client 86
may 87
need 88
constant 89
issu 90
imposs 91
test 92
soy 93
definit 94
thing 95
indent 96
framework 97
would 98
appli 99
reduc 100
renam 101
final 102
privat 103
t

In [112]:
for idx, commit in enumerate(df['commit_message'].values): 
    commit = commit.split()
    for word in commit: 
        if word in key_words_features_dict.keys(): 
            key_words_features[idx, key_words_features_dict[word]] = 1
np.set_printoptions(precision=2)
print(key_words_features[1, :])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.